<a href="https://colab.research.google.com/github/bhavepurva/Sarcasm-detector/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Determine whether news headline is sarcastic or not


getting data into colab

In [ ]:
#downloaded a zip file, uploaded to drive, mount drive on colab
# use below command to unzip it, from and to path

!unzip -uq "/content/drive/My Drive/archive.zip" -d "/content/sarcasms.json"



our data is in json format so we need to convert it into python format

In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

#load json file into a data list
with open('/content/sarcasms.json/Sarcasm_Headlines_Dataset.json',"r") as f:
  data = [json.loads(line) for line in f]

print(data[0])

#creates three separate lists
sentences=[i["headline"] for i in data]
labels=[i["is_sarcastic"] for i in data]
articles=[i["article_link"] for i in data]


{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}


now that we have our list of sentences ready we can start tokenizing the that is preprocessing our data specifically sentences list.

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


splitting data into training and testing sets

In [ ]:
training_sentences=sentences[:training_size]
testing_sentences=sentences[training_size:]

training_labels=labels[:training_size]
testing_labels=labels[training_size:]

tokenizing and padding our sentences


In [ ]:
token=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
token.fit_on_texts(training_sentences)
word_index=token.word_index

training_sequences=token.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

testing_sequences=token.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

for tensorflow version 2 


In [ ]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

defining and training our model


In [ ]:
#top layer is embedding
#neural net model
model=tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(24,activation="relu"),
  tf.keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
num_epochs=30
history=model.fit(training_padded,training_labels,epochs=num_epochs,validation_data=(testing_padded,testing_labels),verbose=2)

testing model with new sentences- first tokenize and pad new sentences and then predict

In [ ]:
sentence=["granny starting to fear spiders in the garden might be real","top snake handler leaves sinking huckabee campaign","how high can this be","the weather today is bright and shiny","frightened by boar market"]
sequences=token.texts_to_sequences(sentence)
padded=pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
predictions=model.predict(padded)
for i in range(len(predictions)):

  print(predictions[i])
  if predictions[i]<0.50:
    print("not sarcastic")
  else:
    print("sarcastic")

[0.9761783]
sarcastic
[0.9986209]
sarcastic
[0.0004023]
not sarcastic
[3.2845583e-05]
not sarcastic
[0.99661]
sarcastic
